In [19]:
from asammdf import MDF
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from pprint import pprint
import sys
import logging
import time


sys.path.append(str(Path("D:/utils/grafana-log-viewer").resolve()))

from decoder.D65.send_d65_data import *

logging.basicConfig(level=logging.INFO, format='%(message)s')

start_date= datetime(2025, 8, 1, tzinfo=ZoneInfo("America/Vancouver"))
end_date = start_date + timedelta(days=30)
files: list[CSVContent] = [f for f in get_all_d65_cancloud_files(start=start_date, end=end_date) if f[1] == 'Lower']

print(f"Total files to process: {len(files)}")

[2025-11-14 17:31:25,373] send_d65_data.py:609  📁 Reading CANCloud files from D:\d65files ...
[2025-11-14 17:31:25,948] send_d65_data.py:617  ✔️  [CANCloud] Found 271 files in 0.574s


Total files to process: 130


In [20]:
def process(f: Path, pgn: int) -> tuple[bool, Path]:
    single_mdf = MDF(f, process_bus_logging=False)
    df = single_mdf.to_dataframe()
    df.columns = [col.replace("CAN_DataFrame.", "") for col in df.columns]
    def check_column(col_name: str) -> bool:
        if col_name not in df.columns:
            return False

        df[col_name + "hex"] = df[col_name].apply(lambda x: hex(x))
        ids = df[col_name + "hex"].unique().tolist()
        ids.sort()
        return (pgn << 8) in [int(_id,16) & 0x00FFFF00 for _id in ids]
    
    for col in ["ID"] + [c for c in df.columns if c.startswith("ID_")]:
        if check_column(col):
            return (True, f)

    return (False, f)

test_pgn = 0xFF18
ts = time.time()
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process, f, test_pgn) for f, _, _ in files]
    _found_list = []
    last_list_size = 0
    for i, future in enumerate(as_completed(futures)):
        found, _file = future.result()
        new_ts = time.time()
        if new_ts - ts > 5:
            ts = new_ts
            if last_list_size != len(_found_list):
                last_list_size = len(_found_list)
                logging.info(f"Progress: {i}/{len(files)} files processed, {_found_list} found so far.")
            else:
                logging.info(f"Progress: {i}/{len(files)} files processed.")
        if found:
            logging.info(f"[{i:03d}] ✅ Found in {shortpath(_file)}")
            _found_list.append(_file)

logging.info(f"Total files containing 0x{test_pgn:08X}: {len(_found_list)}")


[2025-11-14 17:31:36,428] 2209917334.py:35 Progress: 2/130 files processed.
[2025-11-14 17:31:43,250] 2209917334.py:35 Progress: 6/130 files processed.
[2025-11-14 17:31:52,925] 2209917334.py:35 Progress: 8/130 files processed.
[2025-11-14 17:31:59,579] 2209917334.py:35 Progress: 9/130 files processed.
[2025-11-14 17:32:05,167] 2209917334.py:35 Progress: 11/130 files processed.
[2025-11-14 17:32:10,903] 2209917334.py:35 Progress: 16/130 files processed.
[2025-11-14 17:32:18,935] 2209917334.py:35 Progress: 20/130 files processed.
[2025-11-14 17:32:25,254] 2209917334.py:35 Progress: 22/130 files processed.
[2025-11-14 17:32:33,251] 2209917334.py:35 Progress: 26/130 files processed.
[2025-11-14 17:32:38,985] 2209917334.py:35 Progress: 30/130 files processed.
[2025-11-14 17:32:44,249] 2209917334.py:35 Progress: 34/130 files processed.
[2025-11-14 17:32:49,370] 2209917334.py:35 Progress: 39/130 files processed.
[2025-11-14 17:32:58,719] 2209917334.py:35 Progress: 44/130 files processed.
[20